In [0]:
%python
%pip install py4j setuptools
%restart_python

In [0]:
install.packages("reticulate")
install.packages("sparklyr")
install.packages("pysparklyr")

library(tidyverse)
library(sparklyr)
library(reticulate)
library(pysparklyr)

In [0]:
os <- import("os")
# use_python("/usr/local/lib/python3.12")
print(os$sys$executable)
# use_python("/databricks/python3/bin/python3")

In [0]:
library(sparklyr)
library(dplyr)

# conexao
sc <- spark_connect(method = "databricks")

In [0]:
# exemplo de escrita de delta table
mtcars_id <- mtcars %>% rownames_to_column("car_id")
mtcars_tbl <- copy_to(sc, mtcars_id, "mtcars_id", overwrite = TRUE)
spark_write_table(mtcars_tbl, "mtcars_id", mode = "overwrite")

In [0]:
mtcars_db <- tbl(sc, "mtcars_id")

mtcars_db

In [0]:
fs <- import("databricks.feature_engineering")
fe <- fs$FeatureEngineeringClient()

mtcars_pyspark <- pysparklyr_session$table("default.databricks_asn.mtcars_id")

In [0]:
pyspark <- import("pyspark")
pysparklyr_session <- py$spark
mtcars_pyspark <- pysparklyr_session$read$table("databricks_asn.default.mtcars_id")

fe$create_table(
    name="databricks_asn.default.mtcars",
    primary_keys=c("car_id"),
    df=mtcars_pyspark,
    description="MTCARS do R"
)

In [0]:
%python

spark.read.table("databricks_asn.default.mtcars_id")

In [0]:
%python
from databricks.feature_store import FeatureStoreClient

# Initialize the Feature Store client
fs = FeatureStoreClient()

# Load data into a DataFrame
data = spark.read.format("csv").option("header", "true").load("/path/to/your/data.csv")

# Define the feature table name
feature_table_name = "your_catalog.your_schema.your_feature_table"

# Write the DataFrame to the feature store
fs.create_table(
    name=feature_table_name,
    primary_keys=["primary_key_column"],
    df=data,
    schema=data.schema,
    description="Description of your feature table"
)